In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import pandas as pd
import pickle
from functools import reduce

# Funciones para guardar cookies

In [16]:
def save_cookie(driver, path):
    with open(path, 'wb') as filehandler:
        pickle.dump(driver.get_cookies(), filehandler)

def load_cookie(driver, path):
     with open(path, 'rb') as cookiesfile:
        cookies = pickle.load(cookiesfile)
        for cookie in cookies:
            driver.add_cookie(cookie)

## Definiendo rutas de interes

In [54]:
places = {"cdmx":"%5B%22mx%3A5921%22%5D"}
keywords = ["ceo"]
login_path = "https://www.linkedin.com"
username = "cgonzalez@wiggi.mx"
passwords = "KPn=y9Ku"
cookie_name = "cgonzalez_linkedin.pkl"

In [41]:
def _search_people_link(place, keywords, num_page):
    keywords_string = reduce(lambda a,b: "{} O {}".format(a, b), keywords)
    keywords_string = keywords_string.replace(" ","%20")
    return "https://www.linkedin.com/search/results/people/?facetGeoRegion={}&keywords={}&origin=FACETED_SEARCH&page={}".format(place, keywords_string, num_page)

In [45]:
search_people_link(places["cdmx"], keywords, 1)

'https://www.linkedin.com/search/results/people/?facetGeoRegion=%5B%22mx%3A5921%22%5D&keywords=ceo%20O%20CEO%20O%20CTO&origin=FACETED_SEARCH&page=1'

## Guardando cookies tras login

In [47]:
driver = webdriver.Firefox()
driver.implicitly_wait(30)
document = driver.get(login_path)

In [49]:
# Despues de hacer login
#save_cookie(driver, "cgonzalez_linkedin.pkl")

# Ejemplo buscando en la primera pagina

In [55]:
#launch url
url = search_people_link(places["cdmx"], keywords, 1)
# create a new Firefox session
driver = webdriver.Firefox()
driver.implicitly_wait(30)
driver.get(login_path)
load_cookie(driver, cookie_name)
driver.implicitly_wait(500)
driver.get(url)
# python_button = driver.find_element_by_id('MainContent_uxLevel1_Agencies_uxAgencyBtn_33') #FHSU
# python_button.click() #click fhsu li

In [13]:
soup_level1 = BeautifulSoup(driver.page_source, 'lxml')

In [11]:
soup_level1

<html class="artdeco" lang="en"><head>
<script type="application/javascript">!function(i,n){void 0!==i.addEventListener&&void 0!==i.hidden&&(n.liVisibilityChangeListener=function(){i.hidden&&(n.liHasWindowHidden=!0)},i.addEventListener("visibilitychange",n.liVisibilityChangeListener))}(document,window);</script>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<title>LinkedIn</title>
<meta content="" name="description"/>
<meta content="width=device-width, initial-scale=1.0, minimum-scale=1.0" name="viewport"/>
<meta content="#0077B5" name="theme-color"/>
<meta content="true" name="is-http2"/>
<link href="/voyager/manifest.json" rel="manifest"/>
<meta content="%7B%22modulePrefix%22%3A%22extended%22%2C%22environment%22%3A%22production%22%2C%22ember-a11y-testing%22%3A%7B%22componentOptions%22%3A%7B%22turnAuditOff%22%3Atrue%2C%22excludeAxeCore%22%3Atrue%7D%7D%2C%22lix%22%3A%7B%22lixKeys%22%3A%5B%22voyager.iac.web.abi-remove-results-controller%22%2C%22voyager.i

In [24]:
headers = {
            'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.90 Safari/537.36'
}
companyurls = ['https://www.linkedin.com/company/tata-consultancy-services']
url = companyurls[0]
print ("Fetching :",url)     
response = requests.get(url, headers = headers,verify=False)

Fetching : https://www.linkedin.com/company/tata-consultancy-services


In [26]:
response.

b'<html><head>\n<script type="text/javascript">\nwindow.onload = function() {\n  // Parse the tracking code from cookies.\n  var trk = "bf";\n  var trkInfo = "bf";\n  var cookies = document.cookie.split("; ");\n  for (var i = 0; i < cookies.length; ++i) {\n    if ((cookies[i].indexOf("trkCode=") == 0) && (cookies[i].length > 8)) {\n      trk = cookies[i].substring(8);\n    }\n    else if ((cookies[i].indexOf("trkInfo=") == 0) && (cookies[i].length > 8)) {\n      trkInfo = cookies[i].substring(8);\n    }\n  }\n\n  if (window.location.protocol == "http:") {\n    // If "sl" cookie is set, redirect to https.\n    for (var i = 0; i < cookies.length; ++i) {\n      if ((cookies[i].indexOf("sl=") == 0) && (cookies[i].length > 3)) {\n        window.location.href = "https:" + window.location.href.substring(window.location.protocol.length);\n        return;\n      }\n    }\n  }\n\n  // Get the new domain. For international domains such as\n  // fr.linkedin.com, we convert it to www.linkedin.com\n

In [22]:
from lxml import html
import csv, os, json
import requests
from time import sleep

def linkedin_companies_parser(url):
    for i in range(5):
        try:
            headers = {
                        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.80 Safari/537.36'
                                        
            }
            print( "Fetching :",url     )
            response = requests.get(url, headers = headers,verify=False)
            formatted_response = response.content.replace('<!--', '').replace('-->', '')
            doc = html.fromstring(formatted_response)
            return doc
            datafrom_xpath = doc.xpath('//code[@id="stream-promo-top-bar-embed-id-content"]//text()')
            content_about = doc.xpath('//code[@id="stream-about-section-embed-id-content"]')
            if not content_about:
                content_about = doc.xpath('//code[@id="stream-footer-embed-id-content"]')
            if content_about:
                pass
                # json_text = content_about[0].html_content().replace('<code id="stream-footer-embed-id-content"><!--','').replace('<code id="stream-about-section-embed-id-content"><!--','').replace('--></code>','')
            
            if datafrom_xpath:
                try:
                    json_formatted_data = json.loads(datafrom_xpath[0])
                    company_name = json_formatted_data['companyName'] if 'companyName' in json_formatted_data.keys() else None
                    size = json_formatted_data['size'] if 'size' in json_formatted_data.keys() else None
                    industry = json_formatted_data['industry'] if 'industry' in json_formatted_data.keys() else None
                    description = json_formatted_data['description'] if 'description' in json_formatted_data.keys() else None
                    follower_count = json_formatted_data['followerCount'] if 'followerCount' in json_formatted_data.keys() else None
                    year_founded = json_formatted_data['yearFounded'] if 'yearFounded' in json_formatted_data.keys() else None
                    website = json_formatted_data['website'] if 'website' in json_formatted_data.keys() else None
                    type = json_formatted_data['companyType'] if 'companyType' in json_formatted_data.keys() else None
                    specialities = json_formatted_data['specialties'] if 'specialties' in json_formatted_data.keys() else None

                    if "headquarters" in json_formatted_data.keys():
                        city = json_formatted_data["headquarters"]['city'] if 'city' in json_formatted_data["headquarters"].keys() else None
                        country = json_formatted_data["headquarters"]['country'] if 'country' in json_formatted_data['headquarters'].keys() else None
                        state = json_formatted_data["headquarters"]['state'] if 'state' in json_formatted_data['headquarters'].keys() else None
                        street1 = json_formatted_data["headquarters"]['street1'] if 'street1' in json_formatted_data['headquarters'].keys() else None
                        street2 = json_formatted_data["headquarters"]['street2'] if 'street2' in json_formatted_data['headquarters'].keys() else None
                        zip = json_formatted_data["headquarters"]['zip'] if 'zip' in json_formatted_data['headquarters'].keys() else None
                        street = street1 + ', ' + street2
                    else:
                        city = None
                        country = None
                        state = None
                        street1 = None
                        street2 = None
                        street = None
                        zip = None

                    data = {
                                'company_name': company_name,
                                'size': size,
                                'industry': industry,
                                'description': description,
                                'follower_count': follower_count,
                                'founded': year_founded,
                                'website': website,
                                'type': type,
                                'specialities': specialities,
                                'city': city,
                                'country': country,
                                'state': state,
                                'street': street,
                                'zip': zip,
                                'url': url
                            }
                    print(data)
                    return data
                except:
                    print("cant parse page", url)

            # Retry in case of captcha or login page redirection
            if len(response.content) < 2000 or "trk=login_reg_redirect" in url:
                if response.status_code == 404:
                    print( "linkedin page not found")
                else:
                    print('redirecting to login page or captcha found')
        except :
            print("retrying :",url)

def readurls():
    companyurls = ['https://www.linkedin.com/company/tata-consultancy-services']
    extracted_data = []
    for url in companyurls:
        extracted_data.append(linkedin_companies_parser(url))
        f = open('data.json', 'w')
        json.dump(extracted_data, f, indent=4)

In [23]:
a = readurls()

Fetching : https://www.linkedin.com/company/tata-consultancy-services
retrying : https://www.linkedin.com/company/tata-consultancy-services
Fetching : https://www.linkedin.com/company/tata-consultancy-services
retrying : https://www.linkedin.com/company/tata-consultancy-services
Fetching : https://www.linkedin.com/company/tata-consultancy-services
retrying : https://www.linkedin.com/company/tata-consultancy-services
Fetching : https://www.linkedin.com/company/tata-consultancy-services
retrying : https://www.linkedin.com/company/tata-consultancy-services
Fetching : https://www.linkedin.com/company/tata-consultancy-services
retrying : https://www.linkedin.com/company/tata-consultancy-services


In [ ]:
respon

In [8]:
linkedin_companies_parser()

<function __main__.linkedin_companies_parser(url)>

In [10]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)